In [1]:
#Todos os imports necessários para aplicação.
import os, shutil
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext
from os.path import isfile, join
from os import listdir

In [2]:
#Seta variaveis do Data Lake
LANDED = 'C:/Users/970775/Documents/ProjectsPhyton/Bigdata/Projeto/landed/'
RAW = 'C:/Users/970775/Documents/ProjectsPhyton/Bigdata/Projeto/raw/'
RAWBKP = 'C:/Users/970775/Documents/ProjectsPhyton/Bigdata/Projeto/raw/bkp/'
MODELED = 'C:/Users/970775/Documents/ProjectsPhyton/Bigdata/Projeto/modeled/'

In [3]:
builder = SparkSession.builder.appName("SAVE PARKET")
builder.config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
builder.config("spark.speculation", "false")
builder.config("spark.sql.parquet.compression.codec", "gzip")

spark = builder.getOrCreate()
sql = SQLContext(spark.sparkContext)
spark


In [17]:
#variavel
csvfile = "infracao_anos.csv"

#Funcoes
def create_parquet():
    # read csv
    print("Starting wait ...")
    df = spark.read.csv(RAW + csvfile, sep=";", header='TRUE').select("datainfracao","horainfracao","infracao","localcometimento","agenteequipamento","descricaoinfracao")
    #save 
    df.write.parquet(MODELED + parquetfile, compression='gzip')
    print('Finished ..')
    df.show()
    
def create_csv_full():
    print("Union files")
    csv = [pd.read_csv(f, sep=';', header=None).apply(lambda x: x.astype(str).str.lower()) for f in arquivos]
    full_csv = pd.concat(csv, sort=False)
    full_csv.to_csv( RAW + csvfile, index=False, encoding='utf-8-sig', sep=';', header=None)
    print('Finished ..')
    
def move_files():
    print("movendo arquivos .. ")
    for fm in arquivos:
        shutil.move(fm, RAWBKP)
        print("arquivos movidos")

def define_acao():
    arquivos = [f for f in listdir(RAW) if (isfile(join(RAW, f)) and f.endswith('.csv')) ]
    if arquivos.count(arquivos) > 1:
        print("Encontrado mais de 1 arquivo \nRealizando o tratamento de unificação .....")
        create_parquet()
        move_files()
    
    else:
    
        print("Encontrado 1 arquivo \n\n\nGravando parquet ....")
        create_parquet()
        move_files()
    

In [18]:
#Verifica arquivos na pasta
arquivos = [f for f in listdir(RAW) if (isfile(join(RAW, f)) and f.endswith('.csv')) ]
arquivos

['infracao_anos.csv']

In [19]:
#Entrar com o nome do arquivo 
parquetfile = input("Digite o nome do arquivo :") + ".parquet.gzip"
define_acao()

Digite o nome do arquivo : infracao


Encontrado 1 arquivo 


Gravando parquet ....
Starting wait ...
Finished ..
+------------+------------+--------+--------------------+--------------------+--------------------+
|datainfracao|horainfracao|infracao|    localcometimento|   agenteequipamento|   descricaoinfracao|
+------------+------------+--------+--------------------+--------------------+--------------------+
|  2017-01-01|    20:12:24|    5452|avenida conselhei...|código 8 - autos ...|estacionar o veíc...|
|  2017-01-01|    07:38:21|    7455|av. marechal masc...|código 3 - lombad...|transitar em velo...|
|  2017-01-01|    11:15:50|    7463|av. recife, semaf...|código 3 - lombad...|transitar em velo...|
|  2017-01-01|    11:20:34|    7455|av. gov. agamenon...|código 3 - lombad...|transitar em velo...|
|  2017-01-01|    12:40:59|    7463|rua madre de deus...|código 3 - lombad...|transitar em velo...|
|  2017-01-01|    13:03:06|    7463|av. recife, semaf...|código 3 - lombad...|transitar em velo...|
|  2017-01-01|    14:25: